In [8]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import MobileNetV2

# Define paths
train_dir = '../Dataset/Dog&Cat/train'

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,  # More rotation
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],  # Adjust brightness
    fill_mode='nearest',
    validation_split=0.2)  # 20% validation split

# Validation generator (no augmentation)
validation_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Training generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='training')

# Validation generator
validation_generator = validation_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation')

# Load MobileNetV2 as the base model (pretrained on ImageNet)
base_model = MobileNetV2(input_shape=(150, 150, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze the pretrained layers

# Define the model
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),  # Regularization
    Dense(1, activation='sigmoid')  # Binary classification (dog vs cat)
])

# Compile with SGD optimizer (for better generalization)
model.compile(loss='binary_crossentropy',
              optimizer=SGD(learning_rate=0.001, momentum=0.9),
              metrics=['accuracy'])

# Model summary
model.summary()

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)

# Dynamically calculate steps per epoch
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = validation_generator.samples // validation_generator.batch_size

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=[early_stopping, lr_scheduler])

# Evaluate the model
validation_loss, validation_acc = model.evaluate(validation_generator)
print(f"Validation Accuracy: {validation_acc:.2f}")

# Save the trained model
model.save('../Model/Dog_Cat_ImagePredict_model.h5')


Found 447 images belonging to 2 classes.
Found 110 images belonging to 2 classes.


C:\Users\mostm\AppData\Local\Temp\ipykernel_14488\2559895368.py:46: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(input_shape=(150, 150, 3), include_top=False, weights='imagenet')


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 5, 5, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 32000)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 512)            │    16,384,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,643,009 (71.12 MB)

 Trainable params: 16,385,025 (62.50 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 1/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.6232 - loss: 1.2361 - val_accuracy: 0.9062 - val_loss: 0.3745 - learning_rate: 0.0010
Epoch 2/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 92ms/step - accuracy: 0.9062 - loss: 0.2554 - val_accuracy: 0.8542 - val_loss: 0.6001 - learning_rate: 0.0010
Epoch 3/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 9s 659ms/step - accuracy: 0.7684 - loss: 0.8341 - val_accuracy: 0.8750 - val_loss: 0.2949 - learning_rate: 0.0010
Epoch 4/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 144ms/step - accuracy: 0.6562 - loss: 0.8987 - val_accuracy: 0.8750 - val_loss: 0.2672 - learning_rate: 0.0010
Epoch 5/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 11s 828ms/step - accuracy: 0.8567 - loss: 0.3767 - val_accuracy: 0.9062 - val_loss: 0.4072 - learning_rate: 0.0010
Epoch 6/20
 1/13 ━━━━━━━━━━━━━━━━━━━━ 3s 274ms/step - accuracy: 0.8438 - loss: 0.4420
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 100ms/step - accuracy: 0.8438 - loss: 0.4420 - val

Validation Accuracy: 0.92


ลดขนาด Model

In [6]:
import tensorflow as tf

# โหลดโมเดล .h5
model = tf.keras.models.load_model("../Model/Dog_Cat_ImagePredict_model.h5")

# แปลงเป็น TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # ลดขนาดโมเดล
tflite_model = converter.convert()

# บันทึกโมเดลที่ถูกแปลง
with open("../Model/Dog_Cat_Model.tflite", "wb") as f:
    f.write(tflite_model)

print("✅ โมเดลถูกแปลงเป็น .tflite เรียบร้อยแล้ว!")

INFO:tensorflow:Assets written to: C:\Users\mostm\AppData\Local\Temp\tmpqtwesx8c\assets


INFO:tensorflow:Assets written to: C:\Users\mostm\AppData\Local\Temp\tmpqtwesx8c\assets


Saved artifact at 'C:\Users\mostm\AppData\Local\Temp\tmpqtwesx8c'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 150, 150, 3), dtype=tf.float32, name='input_layer_10')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  1863712534416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1863712528848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1863712533072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1863712532880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1863712534032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1863712531728: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1863712526928: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1863712531152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1863712530960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1863712530384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  186